## Building a Price Corrector Model

In [ ]:
import pandas as pd

# build price corrector
# find median price for specified refnis and property_type at t0
# find median price for same refnis, property_type at t1
# compute a correction factor dividing median price t1 with median price t0
# multiply the original listing_price by this correction factor to get the corrected price t1

def price_corrector(df,listing_price,refnis_code, property_type,t0_quarter, t1_quarter):
    """
    correct a listing price
    """
    # create the time-series column
    #df['quarter']= df['annee'].astype(str)+'_'+ df['periode']

    # find the median price at time t0

    median_t0_df = df[(df['refnis'] == refnis_code) &
    (df['property_type'] == property_type) &
    (df['quarter']== t0_quarter)]

    print(median_t0_df)

    median_t0 = median_t0_df['prix_median'].values[0]


    # find the median price at time t1
    median_t1_df = df[(df['refnis']==refnis_code) &
    (df['property_type']==property_type) &
    (df['quarter']==t1_quarter)]

    median_t1 = median_t1_df['prix_median'].values[0]

    # calculate the corrected price
    corrected_price = listing_price * (median_t1/median_t0)

    return corrected_price

estate_data = pd.read_csv("../data/clean/normalized_table_1nf.csv")
estate_data['quarter'] = estate_data['annee'].astype(str) + '-' + estate_data['periode']



In [3]:
# write test
# example refnis
example_refnis = estate_data ['refnis'].iloc[0]
example_property_type = estate_data['property_type'].iloc[0]

print(example_refnis, example_property_type)

11001 toute_maison


In [8]:
estate_data.head

<bound method NDFrame.head of         refnis    localite  annee periode  nombre_transactions  prix_median  \
0        11001  AARTSELAAR   2010      Q1                 29.0     252000.0   
1        11001  AARTSELAAR   2010      Q2                 25.0     254000.0   
2        11001  AARTSELAAR   2010      Q3                 21.0     255000.0   
3        11001  AARTSELAAR   2010      Q4                 28.0     245000.0   
4        11001  AARTSELAAR   2011      Q1                 23.0     310000.0   
...        ...         ...    ...     ...                  ...          ...   
127069   93090   VIROINVAL   2021      Q3                  2.0          NaN   
127070   93090   VIROINVAL   2021      Q4                  2.0          NaN   
127071   93090   VIROINVAL   2023      Q2                  2.0          NaN   
127072   93090   VIROINVAL   2023      Q3                  1.0          NaN   
127073   93090   VIROINVAL   2024      Q2                  6.0          NaN   

        prix_premier_

In [10]:

# find two different quarters for the example
example_t0 = estate_data[(estate_data['refnis']==example_refnis)&
(estate_data['property_type']==example_property_type)]['quarter'].dropna().unique()[0]

example_t1 = estate_data[(estate_data['refnis']==example_refnis)&
(estate_data['property_type']==example_property_type)]['quarter'].dropna().unique()[10] # a quarter later in time

print (example_t0, example_t1)


2010-Q1 2012-Q3


In [11]:
# use a hypothetical price
listing_price  =  estate_data[(estate_data['refnis']==example_refnis)&
(estate_data['property_type']==example_property_type) &
(estate_data['quarter'] == example_t0)]['prix_median'].values[0]

print(listing_price)


252000.0


In [14]:
# Calculate the corrected price
corrected_price_example = price_corrector(estate_data,listing_price,example_refnis, example_property_type, example_t0,example_t1)

print(f"Example Price Correction:")
print(f"Refnis: {example_refnis}")
print(f"Property Type: {example_property_type}")
print(f"Listed Price at {example_t0}:{listing_price} €")
print(f"Corrected Price at {example_t1}:{corrected_price_example:.2f} €")

   refnis    localite  annee periode  nombre_transactions  prix_median  \
0   11001  AARTSELAAR   2010      Q1                 29.0     252000.0   

   prix_premier_quartile  prix_troisieme_quartile property_type  quarter  
0               225000.0                 290000.0  toute_maison  2010-Q1  
Example Price Correction:
Refnis: 11001
Property Type: toute_maison
Listed Price at 2010-Q1:252000.0 €
Corrected Price at 2012-Q3:280000.00 €


In [35]:
# Wrap price_corrector as a function with concise example
import pandas as pd

# build price corrector
# find median price for specified refnis and property_type at t0
# find median price for same refnis, property_type at t1
# compute a correction factor dividing median price t1 with median price t0
# multiply the original listing_price by this correction factor to get the corrected price t1

def price_corrector(df,listing_price,refnis_code, property_type,t0_quarter, t1_quarter):
    """
    correct a listing price based on median prices at two different quarters

    Parameters:
    df: pd.Dataframe
        DataFrame containing real estate data with columns 'refnis', 'property_type','quarter', and 'prix_median'
    listing_price : float
        The initial listing price to be adjusted
    refnis_code: str or int
        identifier for the municipality
    property_type : str
        Type of property 'toute_maison','maison_2_3','maison_4_plus', 'apartment'
    t0_quarter : str
        The intial quarter string eg. 2022-Q1
    t1_quarter : str
        The target  quarter to which the price is adjusted

    Returns:
        float
            The price corrected for marlet changes between t0_quarter and t1_quarter
    
    Raises: 
        ValueError
            If median price data for provided quarters is missing

    """
    # create the time-series column
    #df['quarter']= df['annee'].astype(str)+'_'+ df['periode']

    # find the median price at time t0
    median_t0_df = df[(df['refnis'] == refnis_code) &
    (df['property_type'] == property_type) &
    (df['quarter']== t0_quarter)]

    print(median_t0_df)

    if median_t0_df.empty:
        raise ValueError(f"No median price data for refnis = {refnis_code},property_type={property_type}, quarter={t0_quarter}")
    
    
    median_t0 = median_t0_df['prix_median'].values[0]


    # find the median price at time t1
    median_t1_df = df[(df['refnis']==refnis_code) &
    (df['property_type']==property_type) &
    (df['quarter']==t1_quarter)]

    if median_t1_df.empty:
        raise ValueError(f"No median price data for refnis = {refnis_code},property_type={property_type}, quarter={t1_quarter}")

    median_t1 = median_t1_df['prix_median'].values[0]

    # calculate the corrected price using ratio of median prices
    corrected_price = listing_price * (median_t1/median_t0)

    return corrected_price

if __name__ == "__main__":
    # Example usage:
    estate_data = pd.read_csv("../data/clean/normalized_table_1nf.csv")
    
    # create the quarter column as combined string of 'annee' and 'periode'
    estate_data['quarter'] = estate_data['annee'].astype(str) + '-' + estate_data['periode']

    # Select example refnins and property type from the dataset
    example_refnis = estate_data ['refnis'].iloc[0]
    example_property_type = estate_data['property_type'].iloc[0]

    #print(example_refnis, example_property_type)

    # find two different quarters for the example
    example_t0 = estate_data[(estate_data['refnis']==example_refnis)&
    (estate_data['property_type']==example_property_type)]['quarter'].dropna().unique()[0]

    example_t1 = estate_data[(estate_data['refnis']==example_refnis)&
    (estate_data['property_type']==example_property_type)]['quarter'].dropna().unique()[10] # a quarter later in time

    #print (example_t0, example_t1)

    # use a hypothetical price
    listing_price  =  estate_data[(estate_data['refnis']==example_refnis)&
    (estate_data['property_type']==example_property_type) &
    (estate_data['quarter'] == example_t0)]['prix_median'].values[0]

    print(listing_price)

    # Calculate the corrected price
    corrected_price_example = price_corrector(estate_data,listing_price,example_refnis, example_property_type, example_t0,example_t1)

    print(f"Example Price Correction:")
    print(f"Refnis: {example_refnis}")
    print(f"Property Type: {example_property_type}")
    print(f"Listed Price at {example_t0}:{listing_price} €")
    print(f"Corrected Price at {example_t1}:{corrected_price_example:.2f} €")




252000.0
   refnis    localite  annee periode  nombre_transactions  prix_median  \
0   11001  AARTSELAAR   2010      Q1                 29.0     252000.0   

   prix_premier_quartile  prix_troisieme_quartile property_type  quarter  
0               225000.0                 290000.0  toute_maison  2010-Q1  
Example Price Correction:
Refnis: 11001
Property Type: toute_maison
Listed Price at 2010-Q1:252000.0 €
Corrected Price at 2012-Q3:280000.00 €


## Forecasting Using Machine Learning

In [1]:
# LighGBM
# Linear Regression

import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from scipy.stats import randint as sp_randint, uniform as sp_uniform

In [29]:
# forecasting models for quarters beyond data
# train on 2010 to 2023
# predict on 2024

# read data
df_real_estate = pd.read_csv("../data/clean/normalized_table_1nf.csv")

# data columns
print(df_real_estate.columns)

#target variable prix_median
# features annee, periode, refnis, localite, property_type

df = df_real_estate[['annee','periode','refnis','property_type','prix_median']].copy()

# Create time-series index
# YYYY-MM-DD
# take first day for consistency

df['date']=df.apply(
    lambda row: pd.to_datetime(f"{int(row['annee'])}-{int(row['periode'].replace('Q', ''))*3-2}-01"),
    axis=1
)

# feature engineering

#create log and rollng features
# sort the data by 'refnis' and 'date' for correct feature creation

df.sort_values(by=['refnis','date'])


# create column for the previous quarter's median price ( Lag 1)
df['prix_median_lag1'] = df.groupby(['refnis', 'property_type'])['prix_median'].shift(1)

# create roling mean feature over the last 4 quarters ( 1year)

df['prix_median_rolling_4q'] = df.groupby(['refnis','property_type'])['prix_median'].transform(
    lambda x: x.rolling(window=4, min_periods=1).mean().shift(1)
)

# one hot encoding for property types
df = pd.get_dummies(df, columns=['property_type'],prefix='type')

# label encode 'refnis' tomake it a numerical feature for LightGBM
le =LabelEncoder()
df['refnis_encoded']=le.fit_transform(df['refnis'])

#drop rows with Nan values in the target /key featurs
df_final = df.dropna().copy()

print("\nfinal data frame head after preparation")
print(df_final.head())
print("\nfinal data frame columns after preparation")
print(df_final.columns)
print("\nfinal data frame shape after preparation")
print(df_final.shape)

 
# split the data
# 2010-2022 training, 2023 validation, 2024 testing
X=df_final[['annee','periode','prix_median_lag1','prix_median_rolling_4q','refnis_encoded']+[col for col in df_final.columns if 'type_' in col]].copy()
y = df_final['prix_median']

# annee and period are useful features
#X['annee'] =X['annee'].astype(int).copy() # adding .copy()
X.loc[:,'annee'] = X['annee'].astype(int)
#X['periode'] = X['periode'].str.replace('Q', '').astype(int).copy() # adding .copy()
X.loc[:,'periode'] = X['periode'].str.replace('Q', '').astype(int)
X['periode'] = pd.to_numeric(X['periode']).astype(int)

# To Do: investigate the warnings

# split data based on years
X_train = X[X['annee']<= 2022] 
y_train = y[X_train.index]

X_val = X[X['annee']== 2023] 
y_val = y[X_val.index]


X_test = X[X['annee']==2024]
y_test = y[X_test.index]


print(f"\nTraining set shape: {X_train.shape}")
print(f"\nValidation set shape: {X_val.shape}")
print(f"\nTesting set shape: {X_test.shape}")


Index(['refnis', 'localite', 'annee', 'periode', 'nombre_transactions',
       'prix_median', 'prix_premier_quartile', 'prix_troisieme_quartile',
       'property_type'],
      dtype='object')

final data frame head after preparation
   annee periode  refnis  prix_median       date  prix_median_lag1  \
1   2010      Q2   11001     254000.0 2010-04-01          252000.0   
2   2010      Q3   11001     255000.0 2010-07-01          254000.0   
3   2010      Q4   11001     245000.0 2010-10-01          255000.0   
4   2011      Q1   11001     310000.0 2011-01-01          245000.0   
5   2011      Q2   11001     293750.0 2011-04-01          310000.0   

   prix_median_rolling_4q  type_apartment  type_maison_2_3  \
1           252000.000000           False            False   
2           253000.000000           False            False   
3           253666.666667           False            False   
4           251500.000000           False            False   
5           266000.000000          

In [30]:
#  train and eveluate model

print("\nTraining Regression model")
lin_reg_model = LinearRegression()
lin_reg_model.fit(X_train,y_train)

# make prediction on the validation set
y_pred_lin_reg_val = lin_reg_model.predict(X_val)
mae_lin_reg_val = mean_absolute_error(y_val,y_pred_lin_reg_val)
print(f"Linear Regression MAE on Validation Set: {mae_lin_reg_val:.2f} EUR")


Training Regression model
Linear Regression MAE on Validation Set: 23952.81 EUR


In [31]:

#LightGBM

#define categorical features for LightGBM
categorical_features = ['refnis_encoded'] +[col for col in X_train.columns if 'type_' in col]

lgbm_model = lgb.LGBMRegressor(
    objective='regression_l1',
    metric='mae',
    n_estimators=1000,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42
)

# Train the LightGBM model with validation set for early stopping
lgbm_model.fit(
    X_train,
    y_train,
    eval_set=[(X_val,y_val)],
    eval_metric='mae',
    callbacks=[lgb.early_stopping(100, verbose=False)],
    categorical_feature=categorical_features
)

y_pred_lgbm_val = lgbm_model.predict(X_val)
mae_lgbm_val = mean_absolute_error(y_val,y_pred_lgbm_val)
print(f"LightGBM MAE on Validation Set:{mae_lgbm_val:.2f} EUR")

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000338 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 945
[LightGBM] [Info] Number of data points in the train set: 37392, number of used features: 9
[LightGBM] [Info] Start training from score 218000.000000
LightGBM MAE on Validation Set:24149.19 EUR


In [32]:
# inference on 2024
y_pred_lin_reg_test = lin_reg_model.predict(X_test)
mae_lin_reg_test = mean_absolute_error(y_test,y_pred_lin_reg_test)
print(f"Linear Regression MAE on Test Set: {mae_lin_reg_test:.2f} EUR")

Linear Regression MAE on Test Set: 26016.85 EUR


In [33]:
y_pred_lgbm_test = lgbm_model.predict(X_test)
mae_lgbm_test = mean_absolute_error(y_test,y_pred_lgbm_test)
print(f"LightGBM MAE on Test Set:{mae_lgbm_test:.2f} EUR")

LightGBM MAE on Test Set:25447.63 EUR


In [ ]:
# analysis
# LGB slightly better than Linear regression
# High price Volatility
# Data Granularity- aggregation at municipality, quarter
# 
# Lag features limitation = derailed by unpredictable market shifts

# improvements
# more features : interest rates, inflation
# demographics
# Policy changes
# hyperparemeter tuning
# feature importance  analysis
# model architectures - RNNs etc
# track model artifacts using mlflow